Вы — аналитик крупного сервиса доставки еды. Вместе с отделом маркетинга вы провели эксперимент, который влияет на увеличение выручки.  

Вариант А: экран оплаты без рекламного предложения "Вам это пригодится"  
Вариант Б: экран оплаты с рекламным предложением "Вам это пригодится"    

Вы провели A/B-тест и получили результаты, которые описаны в файлах orders.csv и visitors.csv.

Задание: Оцените результаты A/B-теста, сформулируйте вывод и добавьте рекомендации для отдела маркетинга.

In [1]:
import pandas as pd
import numpy as np

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

## Загрузка данных

### Таблица 1. Заказы

**transactionId** — идентификатор заказа;  
**visitorId** — идентификатор пользователя, совершившего заказ;  
**date** — дата, когда был совершён заказ;  
**revenue** — выручка заказа;  
**group** — группа A/B-теста, в которую попал заказ.

In [3]:
df = pd.read_csv('orders.csv')
df.head()

,transactionId,visitorId,date,revenue,group
0,3667963787,3312258926,2019-08-15,1650,B
1,2804400009,3642806036,2019-08-15,730,B
2,2961555356,4069496402,2019-08-15,400,A
3,3797467345,1196621759,2019-08-15,9759,B
4,2282983706,2322279887,2019-08-15,2308,B


### Таблица 2. Посетители

**date** — дата;  
**group** — группа A/B-теста;  
**visitors** — количество пользователей в указанную дату в указанной группе A/B-теста

In [4]:
df2 = pd.read_csv('visitors.csv')
df2.head()

,date,group,visitors
0,2019-08-01,A,719
1,2019-08-02,A,619
2,2019-08-03,A,507
3,2019-08-04,A,717
4,2019-08-05,A,756


# Анализ данных

## Часть 1

Сначала проверю, был ли эксперимент корректным и не попал ли пользователь в 2 группы в разные числа 

In [5]:
df['visitorId'].unique().shape[0]

1031

In [6]:
df.groupby(['group', 'visitorId']).size().reset_index(name='count') \
      .sort_values(['count'], ascending=[False])                                         \
      .reset_index(drop = True)

,group,visitorId,count
0,A,4256040402,7
1,B,2458001652,6
2,A,2458001652,5
3,A,2038680547,5
4,B,2378935119,5
...,...,...,...
1084,A,3202540741,1
1085,A,3207734412,1
1086,A,3212572194,1
1087,A,3213223594,1


Всего уникальных пользователей - 1031, но при группировке показывается 1089. Значит часть попала в 2 группы, их нужно выбросить из эксперимента

### Очистка данных

In [7]:
df_group_a = df[df['group'] == 'A']

df_group_a[['visitorId', 'group']][:3] 

,visitorId,group
2,4069496402,A
7,78758296,A
8,295230930,A


In [8]:
df_group_b = df[df['group'] == 'B']

df_group_b[['visitorId', 'group']][:3] 

,visitorId,group
0,3312258926,B
1,3642806036,B
3,1196621759,B


In [9]:
df_users_crossing = df_group_a.merge(df_group_b, left_on='visitorId', right_on='visitorId')
df_users_crossing

,transactionId_x,visitorId,date_x,revenue_x,group_x,transactionId_y,date_y,revenue_y,group_y
0,2961555356,4069496402,2019-08-15,400,A,1473132782,2019-08-12,640,B
1,2223239646,199603092,2019-08-15,3488,A,437656952,2019-08-02,3488,B
2,62794304,199603092,2019-08-22,6976,A,437656952,2019-08-02,3488,B
3,2971973105,199603092,2019-08-27,2790,A,437656952,2019-08-02,3488,B
4,1941707234,199603092,2019-08-12,6438,A,437656952,2019-08-02,3488,B
...,...,...,...,...,...,...,...,...,...
178,3757656646,4186807279,2019-08-14,4990,A,1759418862,2019-08-21,18560,B
179,3757656646,4186807279,2019-08-14,4990,A,380064637,2019-08-27,13740,B
180,3612788481,2712142231,2019-08-14,2490,A,1251767592,2019-08-16,790,B
181,4052155355,2949041841,2019-08-14,190,A,1329499668,2019-08-14,10380,B


In [10]:
df = df[~df.visitorId.isin(df_users_crossing.visitorId)]
df

,transactionId,visitorId,date,revenue,group
0,3667963787,3312258926,2019-08-15,1650,B
1,2804400009,3642806036,2019-08-15,730,B
3,3797467345,1196621759,2019-08-15,9759,B
4,2282983706,2322279887,2019-08-15,2308,B
5,182168103,935554773,2019-08-15,2210,B
...,...,...,...,...,...
1191,3592955527,608641596,2019-08-14,16490,B
1192,2662137336,3733762160,2019-08-14,6490,B
1193,2203539145,370388673,2019-08-14,3190,A
1194,1807773912,573423106,2019-08-14,10550,A


### Проверка

In [11]:
df.groupby(['group', 'visitorId']).size().reset_index(name='count') \
      .sort_values(['count'], ascending=[False])                                         \
      .reset_index(drop = True)

,group,visitorId,count
0,A,2108163459,3
1,A,3908431265,3
2,A,611059232,3
3,B,249864742,3
4,A,3967698036,3
...,...,...,...
968,A,3160565905,1
969,A,3160828523,1
970,A,3161387566,1
971,A,3177606451,1


In [12]:
df['visitorId'].unique().shape[0]

973

### Теперь остались только уникальные пользователи

### Временной интервал

In [13]:
df['date'].sort_values().unique()

array(['2019-08-01', '2019-08-02', '2019-08-03', '2019-08-04',
       '2019-08-05', '2019-08-06', '2019-08-07', '2019-08-08',
       '2019-08-09', '2019-08-10', '2019-08-11', '2019-08-12',
       '2019-08-13', '2019-08-14', '2019-08-15', '2019-08-16',
       '2019-08-17', '2019-08-18', '2019-08-19', '2019-08-20',
       '2019-08-21', '2019-08-22', '2019-08-23', '2019-08-24',
       '2019-08-25', '2019-08-26', '2019-08-27', '2019-08-28',
       '2019-08-29', '2019-08-30', '2019-08-31'], dtype=object)

Вижу, что эксперимент длился с 1 августа по 31 августа 2019

In [14]:
df2[df2['date'] == '2019-08-01']

,date,group,visitors
0,2019-08-01,A,719
31,2019-08-01,B,713


In [15]:
df[df['date'] == '2019-08-01'].shape[0]

40

Мы можем сравнить конверсию в заказ за анализируемый период в категории A и B

### Количество посетителей целевой и контрольной группы

In [16]:
visitors_a = df2.loc[df2['group'] == 'A', 'visitors'].sum()

visitors_a

18736

In [17]:
visitors_b = df2.loc[df2['group'] == 'B', 'visitors'].sum()

visitors_b

18916

### Количество заказов от целевой и контрольной группы

In [18]:
orders_a = df[df['group'] == 'B'].shape[0]

orders_a

548

In [19]:
orders_b = df[df['group'] == 'A'].shape[0]

orders_b

468

### AB - тест
**Нулевая гипотеза**. Конверсия останется прежней после изменения экрана оплаты (задача — опровергнуть гипотезу).  

**Альтернативная гипотеза**. Изменения формы оплаты приведут к повышению конверсии.

**P-value**. Цель нахождения p-value  состоит в том, чтобы определить, отличаются ли наблюдаемые результаты от ожидаемых настолько, чтобы можно было не отвергать «нулевую гипотезу» — гипотезу о том, что между экспериментальными переменными и наблюдаемыми результатами нет никакой связи.

**Уровень значимости a** - уровень значимости показывает, насколько мы уверены в наших результатах. Низкое значение для значимости соответствует низкой вероятности того, что экспериментальные результаты вышли случайными и наоборот.
a = 0.05 или 5%

**Итого**: если p − value меньше нашего фиксированного уровня значимости, на котором мы проверяем гипотезу, то нулевую гипотезу следует отвергнуть, если более – то отвергать нулевую гипотезу оснований нет.

### CR - Conversion rate

In [20]:
cr_a = orders_a / visitors_a
cr_a.round(3)

0.029

In [21]:
cr_b = orders_b / visitors_b
cr_b.round(3)

0.025

https://www.evanmiller.org/ab-testing/chi-squared.html

Нулевая гипотеза отвергается при уровне значимости a == 0.05 (p = 0.00696), это значит, что экран оплаты с призывом к действию конвертирует в заказ лучше

## Часть 2

Кроме конверсии у нас на руках данные по суммам заказов из двух групп, а это значит, что мы можем посмотреть сколько денег нам принесли заказы из двух групп

### Сумма заказов за день

In [22]:
revenue_a = df.loc[df['group'] == 'A', 'revenue'].sum()

revenue_a

3364656

In [23]:
revenue_b = df.loc[df['group'] == 'B', 'revenue'].sum()

revenue_b

5068972

### Средний чек

In [24]:
average_check_a = revenue_a / orders_a

average_check_a

6139.883211678832

In [25]:
average_check_b = revenue_b / orders_b

average_check_b

10831.136752136752

Средний чек так же выше при экране с рекламных предложением

## Вывод: оставляем вариант экрана с рекламных предложением